# Customized Random Forest Classifier Model 

- TODO: implement tqdm.contrib.concurrent for build trees in parallel.

## Fundamentals:

- Based on Random Forest method principles: ensemble of models (decision trees).

- In bootstrap process:

    - the data sampled ensure the balance between classes, for training and validation;

    - the list of features used are randomly sampled (with random number of features and order).
    
- For each tree:

    - fallowing the sequence of a given list of features, the data is splited half/half based on meadian value;
    
    - the splitting process ends when the samples have one only class;
    
    - validation process based on dynamic threshold can discard the tree.
    
- For use the forest:

    - all trees predictions are combined as a vote;
    
    - it is possible to use soft or hard-voting.
    
- Positive side-effects:

    - possible more generalization caused by the combination of overfitted trees, each tree is highly specialized in a smallest and different set of feature;
    
    - robustness for unbalanced and missing data, in case of missing data, the feature could be skipped without degrade the optimization process;
    
    - in prediction process, a missing value could be dealt with a tree replication considering the two possible paths;
    
    - the survived trees have a potential information about feature importance.

### Premises: 
- all features must be numeric.
   
- Is case of categorical data, the splitting is done for each categorical value, creating one branch for each value.

- Author: Israel Oliveira [\[e-mail\]](mailto:'Israel%20Oliveira%20'<prof.israel@gmail.com>)

In [1]:
!pip3 install random-forest-mc

     |█████████                       | 4.4 MB 1.3 MB/s eta 0:00:09^C

ERROR: Operation cancelled by user


In [2]:
%load_ext watermark

In [3]:
import pandas as pd
import numpy as np
from random_forest_mc.model import RandomForestMC
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score
from collections import Counter

In [4]:
# from tqdm import tqdm

# from glob import glob

# import matplotlib.pyplot as plt
# %matplotlib inline
# from matplotlib import rcParams
# from cycler import cycler

# rcParams['figure.figsize'] = 12, 8 # 18, 5
# rcParams['axes.spines.top'] = False
# rcParams['axes.spines.right'] = False
# rcParams['axes.grid'] = True
# rcParams['axes.prop_cycle'] = cycler(color=['#365977'])
# rcParams['lines.linewidth'] = 2.5

# import seaborn as sns
# sns.set_theme()

# pd.set_option("max_columns", None)
# pd.set_option("max_rows", None)
# pd.set_option('display.max_colwidth', None)

from IPython.display import Markdown, display
def md(arg):
    display(Markdown(arg))

# from pandas_profiling import ProfileReport
# #report = ProfileReport(#DataFrame here#, minimal=True)
# #report.to

# import pyarrow.parquet as pq
# #df = pq.ParquetDataset(path_to_folder_with_parquets, filesystem=None).read_pandas().to_pandas()

# import json
# def open_file_json(path,mode='r',var=None):
#     if mode == 'w':
#         with open(path,'w') as f:
#             json.dump(var, f)
#     if mode == 'r':
#         with open(path,'r') as f:
#             return json.load(f)

# import functools
# import operator
# def flat(a):
#     return functools.reduce(operator.iconcat, a, [])

# import json
# from glob import glob
# from typing import NewType


# DictsPathType = NewType("DictsPath", str)


# def open_file_json(path):
#     with open(path, "r") as f:
#         return json.load(f)

# class LoadDicts:
#     def __init__(self, dict_path: DictsPathType = "./data"):
#         Dicts_glob = glob(f"{dict_path}/*.json")
#         self.List = []
#         self.Dict = {}
#         for path_json in Dicts_glob:
#             name = path_json.split("/")[-1].replace(".json", "")
#             self.List.append(name)
#             self.Dict[name] = open_file_json(path_json)
#             setattr(self, name, self.Dict[name])


In [5]:
# Run this cell before close.
%watermark -d --iversion -b -r -g -m -v
!cat /proc/cpuinfo |grep 'model name'|head -n 1 |sed -e 's/model\ name/CPU/'
!free -h |cut -d'i' -f1  |grep -v total

Python implementation: CPython
Python version       : 3.9.7
IPython version      : 7.27.0

Compiler    : GCC 10.2.1 20210110
OS          : Linux
Release     : 5.11.0-7620-generic
Machine     : x86_64
Processor   : 
CPU cores   : 8
Architecture: 64bit

Git hash: c138ee6c0b76908c9be71e54c61d6d3ec979e1d1

Git repo: https://github.com/ysraell/random-forest-mc.git

Git branch: main

pandas: 1.3.2
numpy : 1.21.2

CPU	: Intel(R) Xeon(R) CPU E3-1241 v3 @ 3.50GHz
Mem:            31G
Swap:          4.0G


In [6]:
df = pd.read_csv('/work/data/creditcard_trans_int.csv')
target_col = 'Class'
ds_cols = ['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount']

## How much data for each class:

In [7]:
df[target_col].value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [8]:
# How much for test step:
N_fraud_test = 200
N_truth_test = int(2e4)

# How much for training step:
N_truth_train = int(2e5)
# The remaing data for fraud is all used.

# How much (survived) trees:
T = 128

# Parameters for bootstrap

# Data for each tree:
N_T = int((df[target_col].value_counts()[1] - N_fraud_test)/2) # remaing data for fraud.
N_V = N_T # keep balanced amount for each classe.

# How much features:
n_F_max = len(ds_cols)
n_F_min = n_F_max//2
# From EDA: decision trees with small amount of features could be not so efficient.

# Droped trees limit:
D = 64
# When a total of D trees are dropped, the validation threshold decreases delta_Th, restarting the counting.

# Delta Threshold
delta_th = 0.001
th_start = 0.94
# The validation threshold decreases dynamically as need to selection more specialized tree.

# Metric in validation
M = 'Accuracy'
# This parameter have no effect, only to say which metric is used in validation process.

#Seeds
#split_seeds = [43, 47, 53, 59]
split_seeds = [43]
# One for each experiment.

In [14]:
N_T

146

In [ ]:
# Splitting by unique values of the classes.
df = df[ds_cols+[target_col]]
df_fraud = df.query('Class == 1').reset_index(drop=True).copy()
df_truth = df.query('Class == 0').reset_index(drop=True).copy()

Results = {}
    
# Each seed is a experiment.
for seed in split_seeds:

    # Start the experiment
    df_fraud_train, df_fraud_test = train_test_split(df_fraud, test_size=N_fraud_test, random_state=seed)
    df_truth_train, df_truth_test = train_test_split(df_truth, train_size=N_truth_train, test_size=N_truth_test, random_state=seed)
    df_train = pd.concat([df_fraud_train, df_truth_train]).reset_index(drop=True)
    df_test = pd.concat([df_fraud_test, df_truth_test]).reset_index(drop=True)
    del df_fraud_test, df_truth_test, df_fraud_train, df_truth_train
    
    # Training step
    cls = RandomForestMC(
        n_trees=T, target_col='Class', max_discard_trees=D,
        delta_th=delta_th, th_start=th_start,
        batch_train_pclass=N_T, batch_val_pclass=N_V,
        th_decease_verbose = False
    )
    cls.process_dataset(df_train)
    cls.fitParallel(max_workers=16, thread_parallel_method=False)
    #cls.fit()
    
    # Test step
    probs = cls.testForestProbs(df_test)

    # Save results
    target_test = df_test.Class.to_list()
    Results[seed] = {
        'probs': probs,
        'target_test': target_test
    }

Planting the forest:   0%|          | 0/64 [00:00<?, ?it/s]

In [10]:
cls.survived_scores

[0.8949999999999999,
 0.8849999999999999,
 0.8929999999999999,
 0.9029999999999999,
 0.9029999999999999,
 0.8849999999999999,
 0.8959999999999999,
 0.8799999999999999,
 0.9009999999999999,
 0.8839999999999999,
 0.8889999999999999,
 0.8899999999999999,
 0.8899999999999999,
 0.8969999999999999,
 0.9029999999999999,
 0.8669999999999999,
 0.8759999999999999,
 0.8969999999999999,
 0.8889999999999999,
 0.8929999999999999,
 0.8899999999999999,
 0.9059999999999999,
 0.8869999999999999,
 0.8959999999999999,
 0.8939999999999999,
 0.8649999999999999,
 0.9009999999999999,
 0.8869999999999999,
 0.8889999999999999,
 0.8899999999999999,
 0.8889999999999999,
 0.8979999999999999,
 0.8789999999999999,
 0.8839999999999999,
 0.8909999999999999,
 0.8919999999999999,
 0.8909999999999999,
 0.8989999999999999,
 0.8779999999999999,
 0.8829999999999999,
 0.9059999999999999,
 0.8899999999999999,
 0.8869999999999999,
 0.8829999999999999,
 0.8879999999999999,
 0.8959999999999999,
 0.8939999999999999,
 0.9109999999

16 threads: `128/128 [50:28<00:00, 14.24s/it]`, `512/512 [6:16:13<00:00, 21.88s/it]`

In [ ]:
"With 16 threads: {:.2f}s/tree".format((6*3600+16*60+13)/512)

In [ ]:

# Generates the metrics
data = []
for seed,exp in Results.items():
    target_test = exp['target_test']
    rf_classes = [0, 1]
    df_exp = pd.DataFrame([(d['0'], d['1']) for d in exp['probs']], columns=rf_classes)
    df_exp['pred'] = df_exp[[0, 1]].apply(lambda x: rf_classes[np.argmax(x)], axis=1)
    df_exp['target'] = target_test
    Fraud_True_Sum = df_exp.loc[(df_exp.pred == 1) & (df_exp.target == 1)][1].sum()/sum(df_exp.target == 1)
    Truth_False_Sum = df_exp.loc[(df_exp.pred == 0) & (df_exp.target == 1)][0].sum()/sum(df_exp.target == 1)
    Fraud_False_Sum = df_exp.loc[(df_exp.pred == 1) & (df_exp.target == 0)][1].sum()/sum(df_exp.target == 0)
    F1_M = f1_score(target_test, df_exp['pred'].to_numpy(), average='macro')
    AUC_ROC_M = roc_auc_score(target_test, df_exp['pred'].to_numpy(), average='macro')
    TP_0 = df_exp.loc[(df_exp.pred == 0) & (df_exp.target == 0)].shape[0]/sum(df_exp.target == 0)
    TP_1 = df_exp.loc[(df_exp.pred == 1) & (df_exp.target == 1)].shape[0]/sum(df_exp.target == 1)


    data.append([
        seed, Fraud_True_Sum, Truth_False_Sum, Fraud_False_Sum, F1_M, AUC_ROC_M, TP_0, TP_1
    ])

In [ ]:
columns = ['seed', 'Fraud_True_Sum','Truth_False_Sum', 'Fraud_False_Sum', 'F1_M', 'AUC_ROC_M', 'TP_0', 'TP_1']
df_Results = pd.DataFrame(data, columns=columns)
df_Results.to_csv('/work/data/Results_creditcard_RFMC_5.csv', index=False)

# Results

In [ ]:
md('# T = 128 e D = 512 (delta_th = 0.002)')
#md("With 16 threads: {:.2f}s/tree".format((6*3600+16*60+13)/512))
df_Results

In [ ]:
md('# T = 512 e D = 32')
md("With 16 threads: {:.2f}s/tree".format((6*3600+16*60+13)/512))
df_Results

In [ ]:
md('# T = 128 e D = 16')
md("With 16 threads: {:.2f}s/tree".format((50*60+28)/128))
df_Results = pd.read_csv('/work/data/Results_creditcard_RFMC_3.csv')
df_Results

### Basic statistics for all experiments

In [ ]:
df_Results[df_Results.columns[1:]].describe().loc[['mean', 'std', 'min', 'max']]

# Baseline model metrics

In [ ]:
df_Baseline = pd.read_csv('Results_creditcard_Baseline.csv')

In [ ]:
columns = ['seed', 'n_estimators', 'Fraud_True_Sum','Truth_False_Sum', 'Fraud_False_Sum', 'F1_M', 'AUC_ROC_M', 'TP_0', 'TP_1']
df_Baseline[columns].groupby(['seed','n_estimators']).mean()

In [12]:
Counter([round(x,8) for x in cls.survived_scores])

Counter({0.895: 1,
         0.885: 2,
         0.893: 2,
         0.903: 3,
         0.896: 3,
         0.88: 2,
         0.901: 2,
         0.884: 3,
         0.889: 4,
         0.89: 5,
         0.897: 3,
         0.867: 1,
         0.876: 1,
         0.906: 2,
         0.887: 3,
         0.894: 4,
         0.865: 1,
         0.898: 2,
         0.879: 1,
         0.891: 2,
         0.892: 1,
         0.899: 2,
         0.878: 2,
         0.883: 3,
         0.888: 1,
         0.911: 1,
         0.913: 1,
         0.874: 2,
         0.886: 1,
         0.9: 1,
         0.917: 1,
         0.877: 1})